## Simple chat with document using Knowledge Bases for Amazon Bedrock - RetrieveAndGenerate API

In [147]:
#setup

%pip install --upgrade pip
%pip install --upgrade boto3
%pip install --upgrade botocore
%pip install pypdf
%pip install io

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


In [12]:
import boto3
import pprint
from botocore.client import Config

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})


bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              region_name=region,
                              config=bedrock_config,
                                    )

#model should be enabled via AWS console
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

print(f'region: {region}')
print(f'model: {model_id}')


region: eu-west-2
model: anthropic.claude-3-sonnet-20240229-v1:0


For data, you can either upload the document you want to chat with or point to the Amazon Simple Storage Service (Amazon S3) bucket location that contains your file. We provide you with both options in the notebook. However in both cases, the supported file formats are PDF, MD (Markdown), TXT, DOCX, HTML, CSV, XLS, and XLSX. Make that the file size does not exceed 10 MB and contains no more than 20K tokens. A token is considered to be a unit of text, such as a word, sub-word, number, or symbol, that is processed as a single entity. Due to the preset ingestion token limit, it is recommended to use a file under 10MB. However, a text-heavy file, that is much smaller than 10MB, can potentially breach the token limit.

In [137]:
from IPython.display import display
from ipywidgets import FileUpload

# Create a file upload widget
upload_widget = FileUpload(accept='.pdf,.txt', multiple=False)
display(upload_widget)

FileUpload(value=(), accept='.pdf,.txt', description='Upload')

In [200]:
from pypdf import PdfReader
import io

file = upload_widget.value[0];

if(file.type == 'application/pdf'):
    reader = PdfReader(io.BytesIO(file.content))   
    page_count = 1
    for page in reader.pages:
        text+= f"\npage_{str(page_count)}\n {page.extract_text()}"

    print(f'pdf file. file name: {file.name}, file size: {file.size}, pages: {len(reader.pages)}, text size {len(text)}, avg used tokens: {len(text)/6}')
else:
    text = bytes(file.content).decode('utf-8')
    print(f'text file. file name: {file.name}, file size: {file.size}, text size {len(text)}, avg used tokens: {len(text)/6}')


text file. file name: Patchworks01.txt, file size: 423269, text size 422628, avg used tokens: 70438.0


### RetreiveAndGenerate API for chatting with your document
The code in the below cell, defines a Python function called `retrieveAndGenerate` that takes two optional arguments: `input` (the input text) and `sourceType` (the type of source to use, defaulting to "S3"). It also sets a default value for the `model_id` parameter.

The function constructs an Amazon Resource Name (ARN) for the specified model using the `model_id` and the `REGION` variable.

If the `sourceType` is "S3", the function calls the `retrieve_and_generate` method of the `bedrock_agent_client` object, passing in the input text and a configuration for retrieving and generating from external sources. The configuration specifies that the source is an S3 location, and it provides the S3 URI of the document.

If the `sourceType` is not "S3", the function calls the same `retrieve_and_generate` method, but with a different configuration. In this case, the source is specified as byte content, which includes a file name, content type (application/pdf), and the actual text data.

In [184]:

def retrieveAndGenerate(input, sourceType="S3", model_id = "anthropic.claude-3-sonnet-20240229-v1:0"):
    model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'
    if sourceType=="S3":
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    "sources": [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_s3_uri
                            }
                        }
                    ]
                }
            }
        )
    else:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    "sources": [
                        {
                            "sourceType": sourceType,
                            "byteContent": {
                                "identifier": file.name,
                                "contentType": "application/pdf" if file.type == 'application/pdf' else "text/plain",
                                "data": text,
                                }
                        }
                    ]
                }
            }
        )

If you want to chat with the document by uploading the file use `sourceType` as `BYTE_CONTENT` for pointing it to s3 bucket, use `sourceType` as `S3`.

In [202]:
query1 = "Summarize the document"
query2 = "Get document statistics, including the count of words, sentences, the top 10 characters and words, and other useful information"
response = retrieveAndGenerate(input=query2, sourceType="BYTE_CONTENT")
generated_text = response['output']['text']
pp.pprint(generated_text)

('Here are some key statistics for the provided document:\n'
 '\n'
 'Word Count: 10,524\n'
 'Sentence Count: 1,032\n'
 '\n'
 'Top 10 Most Frequent Characters:\n'
 '1. Space ( ): 11,524 occurrences\n'
 '2. e: 5,524 occurrences \n'
 '3. a: 4,032 occurrences\n'
 '4. t: 3,984 occurrences\n'
 '5. o: 3,584 occurrences\n'
 '6. i: 3,456 occurrences\n'
 '7. n: 3,360 occurrences\n'
 '8. r: 3,216 occurrences\n'
 '9. s: 3,024 occurrences\n'
 '10. l: 2,448 occurrences\n'
 '\n'
 'Top 10 Most Frequent Words:\n'
 '1. the: 1,344 occurrences\n'
 '2. to: 576 occurrences\n'
 '3. and: 528 occurrences\n'
 '4. a: 480 occurrences\n'
 '5. of: 432 occurrences\n'
 '6. you: 384 occurrences\n'
 '7. for: 336 occurrences\n'
 '8. in: 312 occurrences\n'
 '9. is: 288 occurrences\n'
 '10. with: 264 occurrences')


## Citations or source attributions
Lets retrieve the source attribution or citations for the above response.

In [203]:
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
         contexts.append(reference["content"]["text"])

pp.pprint(contexts)

[]
